In [33]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import matplotlib.pyplot as plt

In [34]:
# データの読み込み
data = pd.read_csv('dataframe.csv')
X = data.iloc[:, :-1].values  # 入力データ (スピン配位)
y = data.iloc[:, -1].values   # 出力データ (温度)

In [35]:
# NumPy配列からPyTorchのテンソルに変換
X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.long).view(-1, 1)  # クラス分類なのでラベルをlong型に変換し、次元を(25000, 1)に変更

In [36]:
# ニューラルネットワークモデルの定義
class NeuralNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = torch.tanh(self.fc1(x))
        x = self.fc2(x)           # ソフトマックス関数を適用する前に、クラスのインデックスを表すLong型にしない
        return x

In [37]:
# モデルの初期化
input_size = 100
hidden_size = 64
output_size = 25
model = NeuralNetwork(input_size, hidden_size, output_size)


In [38]:
# KLダイバージェンス損失関数の定義
criterion = nn.KLDivLoss(reduction='batchmean')
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [41]:
# 学習の実行
num_epochs = 7500
train_losses = []

for epoch in range(num_epochs):
    optimizer.zero_grad()       # 勾配を初期化
    outputs = model(X)          # フォワードパス
    loss = criterion(torch.softmax(outputs, dim=1), y)  # ソフトマックス関数を適用してからクラスのインデックスに変換
    loss.backward()            # バックプロパゲーション
    optimizer.step()           # パラメータの更新

    train_losses.append(loss.item())

    if (epoch + 1) % 100 == 0:
        print(f'Epoch [{epoch + 1}/{num_epochs}], Train Loss: {loss.item():.4f}')


RuntimeError: Found dtype Float but expected Long

In [ ]:
# 損失の可視化
plt.plot(range(1, num_epochs + 1), train_losses)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss')
plt.show()